In [1]:
import os
os.listdir()


['.ipynb_checkpoints',
 'orders.csv',
 'restaurants.sql',
 'Untitled.ipynb',
 'users.json']

In [3]:
import pandas as pd

orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
import sqlite3

conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)


In [6]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
df = orders.merge(users, on="user_id", how="left")
final_df = df.merge(restaurants, on="restaurant_id", how="left")

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [8]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [9]:
final_df.shape
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [10]:
gold_df = df[df['membership'] == 'Gold']


In [11]:
city_revenue = gold_df.groupby('city')['total_amount'].sum()


In [12]:
city_revenue.sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [1]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


NameError: name 'df' is not defined

In [16]:
df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name', 'name', 'city', 'membership'],
      dtype='object')

In [15]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("restaurants.db")
restaurants_df = pd.read_sql("SELECT * FROM restaurants", conn)
conn.close()


In [17]:
restaurants_df.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [18]:
df = df.merge(
    restaurants_df,
    on="restaurant_id",
    how="left"
)


In [19]:
df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [20]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [21]:
user_total = df.groupby("user_id")["total_amount"].sum()


In [22]:
(user_total > 1000).sum()


np.int64(2544)

In [23]:
rating_bins = [3.0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']

df['rating_range'] = pd.cut(df['rating'], bins=rating_bins, labels=rating_labels)


In [24]:
df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)


C:\Users\asus\AppData\Local\Temp\ipykernel_17196\3480674029.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)


rating_range
4.6 – 5.0    2197030.75
4.1 – 4.5    1960326.26
3.0 – 3.5    1881754.57
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

In [25]:
df[df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [26]:
df.groupby('cuisine')['restaurant_id'].nunique().sort_values()


cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

In [27]:
df.groupby('cuisine')['total_amount'].sum().sort_values(ascending=False)


cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64

In [28]:
total_orders = len(df)
total_orders


10000

In [30]:
gold_orders = df[df["membership"] == "Gold"].shape[0]
gold_orders


4987

In [31]:
percentage = (gold_orders / total_orders) * 100
round(percentage)


50

In [33]:
order_counts = df.groupby('restaurant_name_y')['order_id'].count()
avg_order_value = df.groupby('restaurant_name_y')['total_amount'].mean()
restaurant_stats = pd.concat(
    [order_counts, avg_order_value],
    axis=1,
    keys=['order_count', 'avg_order_value']
)

# Filter restaurants with < 20 orders
filtered = restaurant_stats[restaurant_stats['order_count'] < 20]

# Sort by highest average order value
filtered.sort_values('avg_order_value', ascending=False).head(10)


,order_count,avg_order_value
restaurant_name_y,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
Restaurant_298,15,989.822000
Restaurant_56,18,989.467222
Restaurant_135,18,988.702222
Restaurant_343,17,986.026471


In [34]:
df.groupby(['membership','cuisine'])['total_amount'].sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [35]:
df['order_date'] = pd.to_datetime(df['order_date'])
df.groupby(df['order_date'].dt.to_period('Q'))['total_amount'].sum()


C:\Users\asus\AppData\Local\Temp\ipykernel_17196\4198287336.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


order_date
2023Q1    1993425.14
2023Q2    1945348.72
2023Q3    2037385.10
2023Q4    2018263.66
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [36]:
gold_orders = df[df["membership"] == "Gold"].shape[0]
gold_orders


4987

In [37]:
hyderabad_revenue = df[df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)


1889367

In [38]:
distinct_users = df["user_id"].nunique()
distinct_users


2883

In [42]:
avg_gold_aov = df[df["membership"] == "Gold"]["total_amount"].mean()
round
(avg_gold_aov, 2)


(np.float64(797.1455564467616), 2)

In [40]:
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [41]:
gold_top_city_orders = df[
    (df["membership"] == "Gold") & (df["city"] == "Chennai")
].shape[0]

gold_top_city_orders


1337